### Configuração (comece por aqui)

In [ ]:
const GOOGLE_API_KEY = '' // SEU ID DA API DO GOOGLE AQUI

In [5]:
import { ChatGoogleGenerativeAI } from 'npm:@langchain/google-genai'

let model = new ChatGoogleGenerativeAI({
    apiKey: GOOGLE_API_KEY,
    model: "gemini-2.5-flash-lite",
});

In [6]:
const display = {
    md: (txt) => Deno.jupyter.display({
      "text/markdown": txt,
    }, { raw: true }),
    txt: (txt) => Deno.jupyter.display({
      "text/plain": txt,
    }, { raw: true }),
    html: (txt) => Deno.jupyter.display({
      "text/markdown": txt,
    }, { raw: true })
}

# Básico de Prompting

## O que é um prompt?

Um prompt é a entrada que a LLM usará para inferir (gerar) uma saída. 

Você pode ir muito longe com prompts básicos, mas a qualidade da saída de uma LLM depende diretamente do quão bom o prompt é.

Prompts geralmente possuem um formato de **pergunta** ou **instrução**, mas frequentemente precisa conter elementos como **contexto** e **exemplos**.

Vamos para o exemplo mais simples de um prompt:

In [4]:
let response = await model.invoke('O céu é')

await display.md(response.text)

O céu é um conceito multifacetado, com significados que se estendem para além da atmosfera terrestre. Aqui estão algumas das interpretações mais comuns:

**No sentido literal/científico:**

*   **A atmosfera terrestre:** É a camada de gases que envolve a Terra. É onde ocorrem fenômenos meteorológicos como nuvens, chuva, vento e relâmpagos. O céu, visto da Terra, é o resultado da interação da luz solar com essa atmosfera.
*   **O espaço exterior:** Quando falamos do céu em um contexto astronômico, nos referimos ao espaço além da atmosfera terrestre, onde estão as estrelas, planetas, galáxias e outros corpos celestes.

**No sentido figurado/cultural/religioso:**

*   **Um lugar de paz e tranquilidade:** Frequentemente associado a um estado de serenidade, harmonia e ausência de problemas.
*   **O paraíso/morada divina:** Em muitas religiões, o céu é concebido como o local onde os justos vão após a morte, um lugar de felicidade eterna em comunhão com uma divindade.
*   **O limite/o máximo:** Pode ser usado para indicar o ponto mais alto possível, o limite de algo. "O céu é o limite" significa que não há restrições.
*   **O firmamento:** A abóbada celeste que vemos durante o dia (azul) e à noite (estrelada).

**Em resumo, o céu pode ser:**

*   **A vastidão azul acima de nós durante o dia.**
*   **O universo escuro pontilhado de estrelas à noite.**
*   **Um ideal de perfeição e felicidade.**
*   **Um destino espiritual.**

Qual dessas definições você tinha em mente? Ou você estava pensando em algo mais?

Você pode observar que, embora o modelo tenha respondido algo no contexto do prompt, se o nosso objetivo é que ele simplesmente complete a frase, o resultado está bem longe de cumprir esta tarefa.

Ou seja, mesmo em um exemplo tão simples quanto esse, fica claro o quão necessário é prover mais contexto ou instruções mais específicas sobre a tarefa em questão.

Esse é o princípio de **Prompt Engineering**.

Vamos tentar:

In [5]:
let response = await model.invoke(`
Apenas complete a frase:
O céu é...
`)

await display.md(response.text)

O céu é... **azul**

Melhor, né?

Essa abordagem de projetar prompts para garantir que resolvam uma tarefa de forma acurada é chamada de **Prompt Engineering**.

---

<br>
<br>
<br>

## Formatos de Prompts

Tentamos um prompt muito simples acima. Um prompt padrão tem o seguinte formato:

```
<Pergunta>?
```

ou

```
<Instrução>
```

Isso pode ser formatado em um formato de resposta a perguntas (Q&A), que é padrão em muitos conjuntos de dados de QA, como segue:

```
Q: <Pergunta>?
A:
```

Ao solicitar como o acima, também chamado de **Zero Shot Prompt**, ou seja, você está solicitando diretamente ao modelo uma resposta sem nenhum exemplo ou demonstração sobre a tarefa que deseja realizar.

Alguns modelos de linguagem grandes têm a capacidade de executar prompts zero-shot, mas isso depende da complexidade e do conhecimento da tarefa em questão.

<br>
<br>
<br>

Dado o formato padrão acima, uma técnica popular e eficaz para solicitação é chamada de **Few Shot Prompt**, onde fornecemos exemplos (ou seja, demonstrações). 

Os prompts de poucos tiros podem ser formatados da seguinte maneira:

```
<Pergunta>?
<Resposta>
<Pergunta>?
<Resposta>
<Pergunta>?
<Resposta>
<Pergunta>?
```

A versão do formato QA ficaria assim:

```
Q: <Pergunta>?
A: <Resposta>
Q: <Pergunta>?
A: <Resposta>
Q: <Pergunta>?
A: <Resposta>
Q: <Pergunta>?
A:
```

<br>
<br>
<br>

Lembre-se de que não é necessário usar o formato QA.
O formato do prompt depende da tarefa em mãos.
Por exemplo, você pode executar uma tarefa de classificação simples e fornecer exemplares que demonstrem a tarefa da seguinte forma:

In [26]:
let response = await model.invoke(`
Isso é incrível! // Positivo
Isto é mau! // Negativo
Uau, esse filme foi radical! // Positivo
Que espetáculo horrível! //
`)

await display.md(response.text)

Negativo

Os prompts de poucos tiros permitem o aprendizado no contexto, que é a capacidade dos modelos de linguagem de aprender tarefas dadas algumas demonstrações.

---

<br>
<br>
<br>

## Elementos de um Prompt

Um prompt pode conter qualquer um dos seguintes componentes:

- **Instrução** - uma tarefa ou instrução específica que você deseja que o modelo execute.

- **Contexto** - pode envolver informações externas ou contexto adicional que pode direcionar o modelo para melhores respostas.

- **Dados de entrada** - é a entrada ou pergunta para a qual estamos interessados em encontrar uma resposta.

- **Indicador de saída** - indica o tipo ou formato da saída.

Nem todos os componentes são necessários para um prompt e o formato depende da tarefa em questão. Abordaremos exemplos mais concretos nos próximos guias.

In [37]:
let response = await model.invoke(`
Classifique o texto entre neutro, negativo ou positivo

Texto: Que espetáculo horrível!.

Sentimento:
`)

await display.md(response.text)

Sentimento: Negativo

Perceba que este exemplo possui instrução, dados de entrada e indicadoor de saída, mas não inclui contexto (como no exemplo de few-shot acima).

---

<br>
<br>
<br>

## Dicas Gerais para Projetar Prompts

Aqui estão |algumas dicas para ter em mente enquanto você projeta seus prompts:

### Comece Simples

- Procure um ambiente limpo de "System prompts" (Ex. OpenAI Playground, Google AI Studio) e crie uma "biblioteca de prompts".
- Você pode começar com prompts simples e continuar adicionando mais elementos e contexto conforme busca resultados melhores. 
- Iterar seu prompt ao longo do tempo é vital por essa razão.
- Vamos passar muitos exemplos onde especificidade, simplicidade e concisão frequentemente darão melhores resultados.

- Quando você tem uma tarefa grande que envolve muitas subtarefas diferentes, pode tentar dividir a tarefa em subtarefas mais simples e continuar construindo conforme obtém resultados melhores.
- Isso evita adicionar muita complexidade ao processo de design de prompts no início.

<br>
<br>
<br>

### A Instrução

- Você pode projetar prompts eficazes para várias tarefas simples usando *comandos* ou *palavras-chave* para instruir o modelo sobre o que deseja alcançar, como "Escreva", "Classifique", "Resuma", "Traduza", "Ordene", etc.

<br>

- Tenha em mente que você também precisa experimentar muito para ver o que funciona melhor.
- Tente diferentes instruções com diferentes palavras-chave, contextos e dados e veja o que funciona melhor para seu caso de uso e tarefa específicos.
- Geralmente, **quanto mais específico e relevante o contexto for para a tarefa que você está tentando realizar, melhor**.

<br>

- É recomendado que você coloque as instruções no início do prompt ou no **system prompt**. 
- Outra recomendação é usar algum **separador** claro como "###" para separar a instrução do contexto.

Por exemplo:

*Prompt:*
```
### Instrução ###
Traduza o texto abaixo para o espanhol:

Texto: "hello!"
```

*Saída:*
```
¡Hola!
```

<br>
<br>
<br>

### Especificidade

- Seja muito **específico** sobre a instrução e tarefa que você quer que o modelo execute.
- Quanto mais descritivo e detalhado o prompt for, melhores serão os resultados.
- Isso é particularmente importante quando você tem um resultado desejado ou estilo de geração que está buscando.
- **Não há tokens ou palavras-chave específicas que levem a resultados melhores.**
- É mais importante ter um bom formato e um prompt descritivo.
- Então geralmente, fornecer exemplos no prompt é a melhor maneira de obter a saída desejada em formatos específicos.

<br>

- Ao projetar prompts, você também deve ter em mente o tamanho do prompt, pois há limitações sobre quão longo o prompt pode ser.
- Pense em quão específico e detalhado você deve ser.
- Incluir muitos detalhes desnecessários não é necessariamente uma boa abordagem.
- Os detalhes devem ser relevantes e contribuir para a tarefa em questão.
- Isso é algo que você precisará experimentar bastante.
- É necessário muita experimentação e iteração para otimizar prompts para suas aplicações.

Agora, vamos tentar um prompt simples para extrair informações específicas de um trecho de texto.

In [40]:
let response = await model.invoke(`
Extraia o nome dos lugares no seguinte texto.

Formato desejado:
Lugar: <lista_de_lugares_separados_por_vírgula>

Entrada: 
"Embora esses desenvolvimentos sejam encorajadores para os pesquisadores, muito ainda é um mistério. 
"Frequentemente temos uma caixa preta entre o cérebro e o efeito que vemos na periferia," diz Henrique Veiga-Fernandes, 
um neuroimunologista no Centro Champalimaud para o Desconhecido em Lisboa. 
"Se queremos usá-lo no contexto terapêutico, precisamos realmente entender o mecanismo.""
`)

await display.md(response.text)

Lugar: Centro Champalimaud para o Desconhecido, Lisboa

<br>
<br>
<br>

### Evite Imprecisão

- Dadas as dicas acima sobre ser detalhado e melhorar o formato, é fácil cair na armadilha de querer ser muito inteligente com os prompts e potencialmente criar descrições imprecisas.
- Frequentemente é melhor ser específico e direto.
- A analogia aqui é muito similar à comunicação eficaz -- quanto mais direta, mais eficaz a mensagem é transmitida.

Por exemplo, você pode estar interessado em aprender o conceito de engenharia de prompts. Você pode tentar algo como:

In [47]:
let response = await model.invoke(`
Explique o conceito de engenharia de prompts. Mantenha a explicação curta, apenas algumas frases, e não seja muito descritivo.
`)

await display.md(response.text)

Engenharia de prompts é a arte e ciência de projetar e otimizar as instruções dadas a modelos de IA para obter resultados desejados. Trata-se de refinar a entrada para guiar o modelo de forma eficaz.

Não está claro no prompt acima quantas frases usar e qual estilo.
Você ainda pode obter boas respostas com os prompts acima, mas o melhor prompt seria aquele que é muito específico, conciso e direto ao ponto.
Algo como:

In [46]:
let response = await model.invoke(`
Use 2-3 frases para explicar o conceito de engenharia de prompts para um estudante do ensino médio.
`)

await display.md(response.text)

Engenharia de prompts é a arte de criar instruções claras e eficazes para modelos de inteligência artificial, como o que estou usando agora. Pense nisso como dar instruções específicas a um amigo robô para que ele entenda exatamente o que você quer e te ajude da melhor forma possível. Quanto melhores forem seus "prompts" (as instruções), melhores serão as respostas que você receberá.

<br>
<br>
<br>

### Fazer ou não fazer?

- Outra dica comum ao projetar prompts **é evitar dizer o que não fazer**, mas dizer o que fazer em vez disso.
- Isso encoraja mais especificidade e foca nos detalhes que levam a boas respostas do modelo.

Aqui está um exemplo de um chatbot de recomendação de filmes falhandoem exatamente o que eu não quero que ele faça por causa de como escrevi a instrução -- focando no que não fazer.

In [59]:
let response = await model.invoke(`
O seguinte é um agente que recomenda filmes para um cliente. NÃO PERGUNTE SOBRE INTERESSES. NÃO PERGUNTE SOBRE INFORMAÇÕES PESSOAIS.

Cliente: Por favor, recomende um filme baseado nos meus interesses.
Agente: 
`)

await display.md(response.text)

Entendido. Com base nos meus dados, posso recomendar um filme que tem tido grande sucesso de crítica e público.

Você gostaria de saber sobre:

*   **"Parasita"**: Um thriller sul-coreano que ganhou o Oscar de Melhor Filme, conhecido por sua abordagem inteligente e chocante das desigualdades sociais.
*   **"Duna"**: Uma épica aventura de ficção científica visualmente deslumbrante, baseada no aclamado romance.
*   **"Nomadland"**: Um drama comovente e contemplativo sobre uma mulher que viaja pelos Estados Unidos após perder tudo na Grande Recessão.

Qual dessas opções lhe parece mais interessante?

Aqui está um prompt melhor:

In [60]:
let response = await model.invoke(`
O seguinte é um agente que recomenda filmes para um cliente.
O agente é responsável por recomendar um filme dos principais filmes em alta globalmente. 
Ele deve se abster de perguntar aos usuários sobre suas preferências e evitar pedir informações pessoais. 
Se o agente não tiver um filme para recomendar, deve responder "Desculpe, não consegui encontrar um filme para recomendar hoje.".

Cliente: Por favor, recomende um filme baseado nos meus interesses.
Agente:
`)

await display.md(response.text)

Desculpe, não consegui encontrar um filme para recomendar hoje.

---

<br>
<br>
<br>

## Exemplos

### Resumo de texto

In [7]:
let response = await model.invoke(`
Declarações de contribuição de autores e agradecimentos em artigos de pesquisa devem declarar de forma clara e específica se, e em que medida, os autores usaram tecnologias de IA como ChatGPT na preparação de seu manuscrito e análise. Eles também devem indicar quais LLMs foram usados. Isso alertará editores e revisores para examinar os manuscritos com mais cuidado em busca de possíveis vieses, imprecisões e creditação inadequada de fontes. Da mesma forma, periódicos científicos devem ser transparentes sobre o uso de LLMs, por exemplo, ao selecionar manuscritos submetidos.

Resuma o texto acima é uma frase.
`)

await display.md(response.text)

Artigos de pesquisa devem declarar o uso de IA, como ChatGPT, e os periódicos devem ser transparentes sobre o uso de LLMs para garantir rigor e creditação adequada.

<br>
<br>
<br>

### Extração de Informação

In [8]:
let response = await model.invoke(`
Declarações de contribuição de autores e agradecimentos em artigos de pesquisa devem declarar de forma clara e específica se, e em que medida, os autores usaram tecnologias de IA como ChatGPT na preparação de seu manuscrito e análise. Eles também devem indicar quais LLMs foram usados. Isso alertará editores e revisores para examinar os manuscritos com mais cuidado em busca de possíveis vieses, imprecisões e creditação inadequada de fontes. Da mesma forma, periódicos científicos devem ser transparentes sobre o uso de LLMs, por exemplo, ao selecionar manuscritos submetidos.

Mencione o produto baseado em modelo de LLM mencionado no parágrafo acima.
`)

await display.md(response.text)

O produto baseado em modelo de LLM mencionado no parágrafo é o **ChatGPT**.

<br>
<br>
<br>

### Q&A

In [11]:
let response = await model.invoke(`
Responda à pergunta com base no contexto abaixo. 
Mantenha a resposta curta e concisa. 
Responda "Não tenho certeza sobre a resposta" se não tiver certeza sobre a resposta.

Contexto: Teplizumab tem suas raízes em uma empresa farmacêutica de Nova Jersey chamada Ortho Pharmaceutical. Lá, cientistas geraram uma versão inicial do anticorpo, apelidado de OKT3. Originalmente obtido de camundongos, a molécula foi capaz de se ligar à superfície das células T e limitar seu potencial de destruição celular. Em 1986, foi aprovado para ajudar a prevenir a rejeição de órgãos após transplantes de rim, tornando-se o primeiro anticorpo terapêutico permitido para uso humano.

Pergunta: De onde o OKT3 foi originalmente obtido?

Resposta:
`)

await display.md(response.text)

Camundongos

<br>
<br>
<br>

### Classificação de Texto

In [13]:
let response = await model.invoke(`
Classifique o texto como neutro, negativo ou positivo. 

Texto: Acho que as férias estão ok.
Sentimento: neutro 

Texto: Acho que a comida estava ok. 
Sentimento:
`)

await display.md(response.text)

neutro

<br>
<br>
<br>

### Geração de Código

In [16]:
let response = await model.invoke(`
/*
Produza somente cóodigo.
Pergunte o nome do usuário e diga "Olá"
*/
`)

await display.md(response.text)

```python
nome = input("Qual é o seu nome? ")
print("Olá", nome)
```

In [17]:
let response = await model.invoke(`
"""
Produza somente cóodigo.
Tabela departments, colunas = [DepartmentId, DepartmentName]
Tabela students, colunas = [DepartmentId, StudentId, StudentName]
Crie uma consulta MySQL para todos os estudantes do Departamento de Ciência da Computação
"""
`)

await display.md(response.text)

```sql
SELECT
    s.StudentId,
    s.StudentName
FROM
    students s
JOIN
    departments d ON s.DepartmentId = d.DepartmentId
WHERE
    d.DepartmentName = 'Ciência da Computação';
```

<br>
<br>
<br>

### Conversas

In [18]:
let response = await model.invoke(`
A seguir está uma conversa com um assistente de pesquisa de IA. O tom do assistente é técnico e científico.

Humano: Olá, quem é você?
IA: Saudações! Sou um assistente de pesquisa de IA. Como posso ajudá-lo hoje?
Humano: Você pode me falar sobre a criação de buracos negros?
IA:
`)

await display.md(response.text)

Absolutamente. A formação de buracos negros é um processo fascinante impulsionado pelos princípios da física estelar e da relatividade geral. Essencialmente, um buraco negro é uma região do espaço-tempo onde a gravidade é tão intensa que nada, nem mesmo partículas e radiação eletromagnética como a luz, consegue escapar de seu horizonte de eventos.

A forma mais comum de formação de buracos negros que observamos e teorizamos é através do colapso gravitacional de estrelas massivas. O processo ocorre da seguinte maneira:

1.  **Fase Estelar:** Estrelas massivas, com massas significativamente maiores que a do nosso Sol (tipicamente mais de 20 a 25 massas solares), gastam sua energia através da fusão nuclear em seus núcleos. Durante a maior parte de suas vidas, a pressão externa gerada pelas reações de fusão contrabalança a força esmagadora da gravidade interna.

2.  **Esgotamento do Combustível Nuclear:** Quando a estrela esgota seu combustível nuclear, principalmente hidrogênio, hélio e elementos mais pesados, as reações de fusão no núcleo começam a diminuir ou cessar.

3.  **Colapso Gravitacional:** Sem a pressão externa para sustentar a estrutura da estrela, a gravidade se torna a força dominante. O núcleo da estrela começa a implodir sob seu próprio peso. Essa implosão é incrivelmente rápida e violenta.

4.  **Formação de uma Supernova:** A implosão do núcleo leva a um "ressalto" e a uma explosão cataclísmica conhecida como supernova. Durante a supernova, as camadas externas da estrela são ejetadas para o espaço, criando espetáculos luminosos que podem brilhar intensamente por semanas ou meses.

5.  **Formação do Buraco Negro (ou Estrela de Nêutrons):** O destino do núcleo remanescente da estrela depende de sua massa:
    *   Se a massa do núcleo remanescente for **inferior a aproximadamente 2-3 massas solares** (o limite de Tolman-Oppenheimer-Volkoff para estrelas de nêutrons), a implosão será interrompida pela pressão de degenerescência de nêutrons, formando uma **estrela de nêutrons**.
    *   No entanto, se a massa do núcleo remanescente for **superior a este limite**, a pressão de degenerescência de nêutrons não é suficiente para deter o colapso gravitacional. A gravidade continua a comprimir a matéria, concentrando uma quantidade imensa de massa em um volume infinitesimal, teoricamente um ponto de densidade infinita chamado **singularidade**.

6.  **Horizonte de Eventos:** À medida que a massa se condensa, o campo gravitacional ao seu redor se torna cada vez mais forte. Uma vez que a massa é comprimida a um determinado raio (o raio de Schwarzschild para um buraco negro não rotativo), a velocidade de escape desse ponto excede a velocidade da luz. Essa fronteira imaginária é o **horizonte de eventos**. Qualquer coisa que cruze o horizonte de eventos está irremediavelmente presa.

Além da formação estelar, existem outras categorias de buracos negros teorizados:

*   **Buracos Negros Supermassivos:** Encontrados nos centros da maioria das galáxias, incluindo a Via Láctea. Suas massas variam de milhões a bilhões de vezes a massa do Sol. Acredita-se que se formem através de processos de acreção contínua de matéria e fusão de buracos negros menores ao longo de bilhões de anos.
*   **Buracos Negros de Massa Intermediária:** Com massas entre buracos negros estelares e supermassivos. Sua formação é menos compreendida, mas podem se formar a partir da fusão de buracos negros estelares em aglomerados estelares densos ou através do colapso direto de nuvens de gás massivas no universo primitivo.
*   **Buracos Negros Primordiais:** Uma classe hipotética de buracos negros que se acredita terem se formado nos primeiros momentos após o Big Bang, quando o universo era extremamente denso e instável. Suas massas poderiam variar amplamente, desde tamanhos subatômicos até massas estelares.

Em resumo, a formação de buracos negros é um resultado direto da lei da gravidade, atuando em condições extremas de densidade e massa, tipicamente observadas no final da vida de estrelas massivas. A relatividade geral fornece o arcabouço teórico para entender a natureza e o comportamento dessas entidades cósmicas.

Existe algum aspecto específico da formação de buracos negros que você gostaria de aprofundar?

In [19]:
let response = await model.invoke(`
"""
A seguir está uma conversa com um assistente de pesquisa de IA. As respostas do assistente devem ser fáceis de entender até mesmo por estudantes do ensino fundamental.

Humano: Olá, quem é você?
IA: Saudações! Sou um assistente de pesquisa de IA. Como posso ajudá-lo hoje?
Humano: Você pode me falar sobre a criação de buracos negros?
IA:
"""
`)

await display.md(response.text)

Claro! Imagine que as estrelas são como bolas de fogo gigantes no espaço. A maioria das estrelas é como o nosso Sol. Mas algumas estrelas são MUITO, MUITO maiores.

Quando uma estrela gigante fica muito velha e esgota todo o seu combustível, ela não consegue mais se sustentar. É como se o ar de um balão estourasse!

Quando isso acontece com estrelas muito, muito grandes, elas encolhem MUITO, MUITO rápido. E quando elas encolhem tão apertado, elas se tornam algo super denso e pesado chamado buraco negro.

Pense em algo que é tão pesado que nem mesmo a luz consegue escapar dele. É por isso que eles são chamados de "buracos negros" – porque não conseguimos vê-los diretamente! É como um ralo gigante no espaço que suga tudo o que chega muito perto.

Então, para criar um buraco negro, você precisa de uma estrela SUPER gigante que morre de uma maneira especial.

Você tem mais alguma pergunta sobre buracos negros?

<br>
<br>
<br>

### Raciocínio

In [20]:
let response = await model.invoke(`
Quanto é 9.000 * 9.000?
`)

await display.md(response.text)

9.000 * 9.000 = **81.000.000**

In [24]:
let response = await model.invoke(`
Somente responda se estou certo e mais nada.

Os números ímpares neste grupo somam um número par: 15, 32, 5, 13, 82, 7, 1.

Estou certo?
`)

await display.md(response.text)

Sim

In [27]:
let response = await model.invoke(`
Os números ímpares neste grupo somam um número par: 15, 32, 5, 13, 82, 7, 1. 

Resolva dividindo o problema em etapas, mas de forma sucinta.
Primeiro, identifique os números ímpares, some-os e indique se o resultado é ímpar ou par.
`)

await display.md(response.text)

**Etapa 1: Identificar os números ímpares**

Os números ímpares no grupo são: 15, 5, 13, 7, 1.

**Etapa 2: Somar os números ímpares**

15 + 5 + 13 + 7 + 1 = 41

**Etapa 3: Indicar se o resultado é ímpar ou par**

O resultado (41) é um número **ímpar**.

<br>
<br>
<br>